In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DTCK_Rental/WHO-COVID-19-global-daily-data.csv')

In [ ]:
data

# Insight từ EDA: Ca nhiễm cao nhất ở Americas/Europe, correlation mạnh giữa cases và deaths (0.8+).


In [ ]:
#Xử lý feature & tiền xử lý cho clustering
numerical = ['Cumulative_cases', 'Cumulative_deaths', 'New_cases', 'New_deaths']
categorical = ['continent']
X = agg_data[numerical + categorical]

In [ ]:
# Tạo target cho classification
agg_data['risk_level'] = pd.qcut(agg_data['Cumulative_deaths'], 3, labels=['Low', 'Medium', 'High'])
y = agg_data['risk_level']

In [ ]:
# Pipeline (giữ nhưng thêm)
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical),
    ('cat', OneHotEncoder(), categorical)
])

In [ ]:
# Clustering (Cải tiến: Elbow + Hierarchical + Silhouette)
X_preprocessed = preprocessor.fit_transform(X)

Giải thích:

Tách ra biến số và biến rời rạc (categorical).

Áp dụng chuẩn hóa (chuẩn z-score cho biến số) và OneHotEncoder cho dữ liệu categorical.

Kết quả là một ma trận feature đã chuẩn hóa, sẵn sàng cho các thuật toán clustering.

In [ ]:
# Elbow method cho KMeans
inertias = []
sil_scores = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_preprocessed)
    inertias.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X_preprocessed, kmeans.labels_))

plt.plot(range(2,10), inertias, marker='o')
plt.title('Elbow Method for Optimal K')
plt.show()

Tính inertia và silhouette score cho K tách cụm từ 2-9, vẽ đồ thị elbow lấy số cụm tối ưu

Đây là biểu đồ elbow (khuỷu tay) dùng trong phân cụm (clustering), tiêu biểu là KMeans.

Các thành phần & ý nghĩa:
1. Trục hoành (x-axis): Số cụm (K)
Các giá trị nguyên từ 2 đến 9, đại diện cho số lượng cụm dùng trong KMeans.

2. Trục tung (y-axis): Inertia (Within-cluster sum of squares)
Giá trị tổng bình phương khoảng cách từ mỗi điểm dữ liệu tới centroid cụm của nó (càng nhỏ, các điểm cùng cụm càng “xít lại”).

Giá trị này giảm dần khi tăng số cụm.

3. Đường plot
Mỗi điểm trên đường ứng với một giá trị K, thể hiện mức độ “nén cụm” đạt được khi đặt K cụm.

4. Elbow (khuỷu tay)
Phần "gấp khúc rõ nhất" của biểu đồ — ở đây nằm quanh K = 3 hoặc K = 4.

Ý nghĩa:

Trước điểm này: Giảm K thêm giúp inertia giảm rất nhanh (chia thêm cụm thực sự giúp nhóm dữ liệu tốt hơn).

Sau điểm này: Tăng tiếp K chỉ giúp inertia giảm chậm (chia thêm cụm không mang lại cải thiện lớn về “chất lượng” phân cụm).

5. Cách chọn số cụm tối ưu (optimal K)
Quan sát điểm "elbow" — K tại điểm gấp khúc mạnh nhất thường được chọn.

Ở biểu đồ này, điểm elbow rõ ràng nhất là tại K = 3 hoặc 4.

In [ ]:
# Chọn k=3 (giả sử từ elbow), run KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
agg_data['kmeans_cluster'] = kmeans.fit_predict(X_preprocessed)
print("Silhouette Score KMeans:", silhouette_score(X_preprocessed, agg_data['kmeans_cluster']))

Chạy KMeans với K tối ưu đã chọn (ở đây là 3). Tính chỉ số silhouette.

In [ ]:
# Hierarchical Clustering
hier = AgglomerativeClustering(n_clusters=3)
agg_data['hier_cluster'] = hier.fit_predict(X_preprocessed)
print("Silhouette Score Hierarchical:", silhouette_score(X_preprocessed, agg_data['hier_cluster']))

Áp dụng phân cụm theo dạng phân cấp/từng bước.


In [ ]:
# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
agg_data['dbscan_cluster'] = dbscan.fit_predict(X_preprocessed)
print("DBSCAN Clusters:", agg_data['dbscan_cluster'].unique())

Phân cụm với DBSCAN, xác định các cụm và noise/outlier.

In [ ]:
# Gắn nhãn cụm với continent
cluster_continent = agg_data.groupby('hier_cluster')['continent'].value_counts().unstack().fillna(0)
print("Cluster vs Continent:\n", cluster_continent)

Dùng PCA giảm còn 2 chiều để trực quan hóa kết quả phân cụm KMeans trên mặt phẳng 2D.

In [ ]:
# Viz clusters
pca = PCA(2)
X_pca = pca.fit_transform(X_preprocessed)
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=agg_data['kmeans_cluster'], palette='viridis')
plt.title('KMeans Clusters (PCA Viz)')
plt.show()

1. Tên biểu đồ
KMeans Clusters (PCA Viz):
Trực quan hóa các cluster (nhóm/cụm) từ thuật toán KMeans, áp lên không gian 2 chiều bằng PCA (Principal Component Analysis).

2. Trục hoành & trục tung
Chỉ là hai thành phần chính (principal components, PC1 & PC2) từ kết quả PCA – không còn là giá trị gốc (như số ca nhiễm, tỷ lệ,...) mà là tổng hợp tuyến tính của nhiều đặc trưng.

3. Điểm và màu sắc
Mỗi dấu chấm: Đại diện cho một quốc gia/điểm dữ liệu.

Màu sắc (theo chú giải legend bên phải):

kmeans_cluster = 0: Một cụm (thường màu tím)

kmeans_cluster = 1: Một cụm khác (thường màu xanh dương/xanh ngọc)

kmeans_cluster = 2: Một cụm nữa (thường màu vàng)

Các điểm cùng màu là cùng cụm do KMeans phân nhóm.

4. Nhận xét hình học
Các cụm nằm khá tách biệt trên mặt phẳng PCA (mặc dù có thể hơi chồng lấn tại tâm).

Một số điểm nằm tách biệt hẳn bên phải → có thể là các quốc gia "đặc biệt" so với phần lớn còn lại (outlier, hoặc những nước có số ca vượt trội hoặc thấp hẳn).

5. Ý nghĩa phân tích
Nhóm lại các nước có đặc điểm dịch tễ COVID-19 "giống nhau" (theo các biến gốc, đã xử lý-noi, chuẩn hóa, one-hot,...).

Nhìn nhanh các cụm có rõ ràng không? Sự khác biệt giữa các quốc gia về tình hình dịch tễ được mô hình phân biệt mạnh – yếu ở mức độ nào (các cụm tách biệt hơn là tốt).

# Insight Clustering (RQ2): Cụm 0: Quốc gia thấp rủi ro (Africa/SEAR), Cụm 1: Trung bình, Cụm 2: Cao (Europe/Americas).

In [ ]:
# Classification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_test_split: Hàm chia bộ dữ liệu thành tập huấn luyện/train (80%) và kiểm thử/test (20%).

X: Tập feature (biến đầu vào).

y: Nhãn phân lớp (ví dụ: ‘risk_level’).

test_size=0.2: Kích thước tập test là 20% tổng dữ liệu.

random_state=42: Đảm bảo kết quả chia là cố định (mọi lần chạy ra cùng kết quả).

In [ ]:
# Pipeline với GridSearch cho RandomForest
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

Pipeline: Chuỗi các bước tiền xử lý rồi huấn luyện model liền nhau.

preprocessor: Tiền xử lý feature (chuẩn hóa, one-hot...).

classifier: Model phân loại – ở đây là RandomForest.

In [ ]:
param_grid = {'classifier__n_estimators': [50, 100], 'classifier__max_depth': [5, 10]}
grid_search = GridSearchCV(rf_pipeline, param_grid, cv=5)

param_grid: Tập các giá trị siêu tham số cần thử cho model:

n_estimators: số lượng cây trong rừng ngẫu nhiên (RF).

max_depth: độ sâu tối đa mỗi cây.

cv=5: 5-fold cross validation.

In [ ]:
grid_search.fit(X_train, y_train)

Tự động thử tất cả combination của n_estimators & max_depth, đánh giá từng mô hình qua CV, rồi trả về mô hình tốt nhất.

In [ ]:
print("Best Params RF:", grid_search.best_params_)
print("Accuracy RF:", accuracy_score(y_test, grid_search.predict(X_test)))

best_params_: Siêu tham số tối ưu vừa tìm được.

accuracy_score: Độ chính xác của model tốt nhất trên tập test.

In [ ]:
# Fix CV
cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5)
print("CV Mean Score RF:", cv_scores.mean())

cross_val_score: Đánh giá chính xác bằng cách train/test 5 lần khác nhau trên toàn bộ data chia đều (tiêu chuẩn hóa đánh giá).

cv_scores.mean(): Giá trị trung bình của các lần cross-validation – thể hiện độ bền của model.

In [ ]:
# Logistic Regression
log_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])
log_pipeline.fit(X_train, y_train)
print("Accuracy Logistic:", accuracy_score(y_test, log_pipeline.predict(X_test)))

Tạo pipeline giống RF nhưng thay RandomForest bằng LogisticRegression.

Huấn luyện, và tính độ chính xác trên tập test như phương pháp phía trên.

In [ ]:
# Confusion Matrix Heatmap
sns.heatmap(confusion_matrix(y_test, grid_search.predict(X_test)), annot=True, cmap='Blues')
plt.title('Confusion Matrix RF')
plt.show()

confusion_matrix: Ma trận đếm số mẫu dự đoán đúng/sai mỗi lớp (thường dùng cho classification).

sns.heatmap: Vẽ ma trận nhầm lẫn dưới dạng heatmap để trực quan hóa.

annot=True: hiển thị số ngay trong từng ô

cmap='Blues': dùng bảng màu xanh.

Giải thích từng thành phần trên hình:
1. Trục ngang & trục dọc
Trục dọc (hàng, axis 0): Lớp thực tế (True label): 0, 1, 2

Trục ngang (cột, axis 1): Lớp dự đoán bởi mô hình (Predicted label): 0, 1, 2

2. Các giá trị trong ma trận
[i, j]: Số trường hợp thực tế là lớp i nhưng dự đoán là lớp j.

3. Ý nghĩa từng giá trị thực tế
Pred=0	Pred=1	Pred=2
True=0	20	0	0
True=1	0	13	0
True=2	1	1	13
Lớp 0: Có 20 mẫu thực tế là 0 → dự đoán đúng cả 20 (ô ); không nhầm lẫn sang các lớp khác.

Lớp 1: Có 13 mẫu thực tế là 1 → dự đoán đúng cả 13 (ô ); không nhầm lẫn.​

Lớp 2: Có 13 mẫu dự đoán đúng (ô ); 1 mẫu lớp 2 lại dự đoán nhầm thành lớp 0, 1 mẫu nhầm thành lớp 1 (ô và ).​

4. Nhận xét hiệu năng
Độ chính xác mô hình khá cao:

Lớp 0 và lớp 1: 100% dự đoán đúng.

Lớp 2: 15 mẫu, dự đoán đúng 13, chỉ 2 mẫu nhầm (1 vào lớp 0, 1 vào lớp 1).

Confusion matrix này cho thấy mô hình dự đoán tuyệt đối tốt với lớp 0 và lớp 1, chỉ có chút nhầm lẫn với lớp 2.

In [ ]:
# ROC Curve
y_prob = grid_search.predict_proba(X_test)
# Giả sử multiclass, dùng one-vs-rest
y_bin = pd.get_dummies(y_test)
roc_auc = roc_auc_score(y_bin, y_prob, multi_class='ovr')
print("ROC AUC RF:", roc_auc)

fpr = dict()
tpr = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_bin.iloc[:, i], y_prob[:, i])
    plt.plot(fpr[i], tpr[i], label=f'Class {i}')
plt.title('ROC Curve RF')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

Giải thích từng dòng:
predict_proba(X_test): Lấy xác suất dự báo cho từng lớp (thay vì nhãn cứng).

get_dummies(y_test): One-hot encoding cho y_test (thành các cột lớp 0, 1, 2).

roc_auc_score(..., multi_class='ovr'): Tính điểm AUC tổng cho mô hình đa lớp (one-vs-rest).

Tạo ROC từng lớp (for i in range(3)) – với fpr (false positive rate) và tpr (true positive rate).

plt.plot(...): Vẽ ROC curve từng lớp.

Hình:
Đường cong ROC cho từng class (Class 0, 1, 2).

Trục X: Sai dương (FPR), Trục Y: Đúng dương (TPR).

Đường càng đi gần góc trái trên càng tốt (model dự đoán càng phân biệt tốt).

In [ ]:
# Báo cáo
print(classification_report(y_test, grid_search.predict(X_test)))

Giải thích:
Tạo báo cáo với precision, recall, f1-score từng lớp và tổng thể.

precision	recall	f1-score	support
High	0.95	1.00	0.98	20
Low	0.93	1.00	0.96	13
Medium	1.00	0.87	0.93	15
macro avg	0.96	0.96	0.96	48
weighted avg	0.96	0.96	0.96	48
Precision: Dự báo đúng chia cho tất cả dự báo cho lớp đó.

Recall: Số đúng chia cho toàn bộ thực tế lớp đó.

F1: Trung bình điều hòa giữa precision & recall.

Support: Số mẫu mỗi lớp.

Nhận xét:
Các lớp đều có precision và recall rất cao (>0.93), F1-score đồng đều – chứng tỏ model cực kỳ tốt.

Đặc biệt: Medium recall thấp nhất (0.87), dễ bị nhầm (xem confusion matrix ở trên).

In [ ]:
# Feature Importance
best_rf = grid_search.best_estimator_.named_steps['classifier']
num_features = numerical
cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical)
all_features = np.concatenate([num_features, cat_features])
importances = best_rf.feature_importances_
importance_df = pd.DataFrame({'Feature': all_features, 'Importance': importances}).sort_values('Importance', ascending=False)
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance RF')
plt.show()

Giải thích:
Lấy các tên feature (cả số và one-hot categorical).

Sử dụng thuộc tính .feature_importances_ của RandomForest để đo lường tầm quan trọng mỗi biến.

Vẽ barplot thể hiện độ quan trọng.

Hình:
Biến Cumulative_deaths quan trọng nhất, tiếp đến là New_deaths, New_cases, Cumulative_cases.

Các biến continent chỉ đóng vai trò nhỏ (tiến gần 0).

Ý nghĩa:
Random Forest dựa chủ yếu vào dữ liệu số ca/to tử vong; vị trí châu lục chỉ ảnh hưởng nhỏ.

# Insight Classification (RQ3): Deaths và Cases là yếu tố quan trọng nhất dự đoán rủi ro. Accuracy ~80-90% tùy data.


In [ ]:
# Viz thêm: Bar chart
sns.barplot(x='continent', y='New_cases', data=agg_data, estimator=np.mean)
plt.title('Average New Cases by Continent')
plt.xticks(rotation=45)
plt.show()

Giải thích:
Vẽ cột trung bình số ca mới từng châu lục.

estimator=np.mean: Tính trung bình từng nhóm.

Hình:
Europe: Có trung bình số ca mới cao nhất, tiếp theo là Americas.

Các châu lục như Africa, Western Pacific thấp hẳn.

Ý nghĩa:
Dữ liệu cho thấy mức độ dịch COVID bùng phát mạnh nhất tại châu Âu.